This demo notebook is to demonstrate initial procedure for generating, displaying, and characterizing defect masks using the DM calibration products pipeline.

From RHL:  

>I don't think you want to bias correct the data (overscan, yes).

>You'd run something like
	runIsr.py /project/shared/comCam --id visit=....  ccd=... --rerun bstalder/XXX -c isr.doBias=False isr.doDark=False isr.doFlat=False
(I see that you're using just --rerun bstalder; that's OK but only allows one rerun).  Add -j 9 or something if the machine is quiet.

>It might actually be easier to run 
	findDefects.py /project/shared/comCam --id visit=....  ccd=... --rerun bstalder/XXX -c makePlots=False -j 9 
that will run the ISR for you with "proper" configs (we need to discuss with Andrés what "proper" should be), and then find the defects.  So if you run that you can then run my notebook to play with the defect finding.

>N.B.:  I just pushed a lost changeset to make this work -- when I said, "I'll just rebase onto master" I forgot that Merlin would be doing a forced push.  I hope I didn't miss anything else!

>Or, contrariwise, you can run the ISR as part of your notebook.  I assume you know how to do that, if not ask!



Example notebook as follows (run on 8/21/2019 with w19_31):

In [1]:
import os
import sys
import numpy as np
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.display.utils as dispUtils 
afwDisplay.setDefaultBackend("matplotlib") 
import matplotlib.pyplot as plt
%matplotlib ipympl 
plt.rcParams['figure.figsize'] = [7, 6]
 # For some reason, to get interactive plots, we need to run this command twice
%matplotlib ipympl


In [2]:
 # For some reason, to get interactive plots, we need to run this command twice
%matplotlib ipympl

Load and initialise the findDefects code (from an RHL branch)

In [13]:
from lsst.cp.pipe.defects import FindDefectsTask 
    
config = FindDefectsTask.ConfigClass() 

findDefectsTask = FindDefectsTask(config=config)

RuntimeError: Must set config.isr.doAddDistortionModel to False for defect finding

Prepare to and then read data

In [ ]:
butler = dafPersist.Butler(os.path.join("/project/shared/comCam", "rerun", "rhl/ 􏰀→defects"))
dataId = dict(detector=194)

In [ ]:
visits = [201904182050253, 201904182051463]

try: 
    exps
except NameError: exps = {}

for visit in visits:
    if visit not in exps:
        exps[visit] = butler.get("postISRCCD", dataId, visit=visit)

Find all the defects

In [ ]:
ret = findDefectsTask.run({}, exps)

defects = ret.defects
#perExposureDefects = ret.perExposureDefects

Visualise the defects

In [ ]:
visit = visits[1]
exp = exps[visit].clone()

exp.mask.addMaskPlane("DEFECT")
afwDisplay.setDefaultMaskPlaneColor("DEFECT", afwDisplay.YELLOW)

defects.maskPixels(exp.maskedImage, "DEFECT") 

disp = afwDisplay.Display(1, reopenPlot=True)
disp.scale('linear', 'zscale')

disp.mtv(exp, title=f"visit={visit} detector={exp.getDetector().getName()}")

In [ ]:
disp.erase()

with disp.Buffering():
    for i, d in enumerate(defects):
        dispUtils.drawBBox(d.getBBox(), display=disp, borderWidth=0.5)

In [ ]:
camera = butler.get("camera") 
for det in camera:
    if "RTM-031" in det.getName(): 
        print(det.getName(), det.getId())